In [6]:
import time
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(30)

In [3]:
model_id = "deepseek-ai/DeepSeek-Prover-V2-7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
formal_statement = """
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
""".strip()

prompt = """
Complete the following Lean 4 code:

```lean4
{}
```

""".strip()

chat = [
  {"role": "user", "content": prompt.format(formal_statement)},
]

inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

<>:9: SyntaxWarning: invalid escape sequence '\%'
<>:9: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipykernel_2116787/143528910.py:9: SyntaxWarning: invalid escape sequence '\%'
  /-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/


In [11]:
start = time.time()
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs))
print(time.time() - start)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


['<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:\n\n```lean4\nimport Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n/-- What is the positive difference between $120\\%$ of 30 and $130\\%$ of 20? Show that it is 10.-/\ntheorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by\n  sorry\n```<｜Assistant｜>```lean4\ntheorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by\n  norm_num [abs_eq_self.mpr, abs_eq_neg_self.mpr, le_of_lt, sub_eq_add_neg, add_assoc]\n  <;> norm_num\n  <;> ring_nf\n  <;> norm_num\n  <;> apply abs_of_nonneg\n  <;> norm_num\n  <;> linarith\n```<｜end▁of▁sentence｜>']
145.98956966400146


In [17]:
print(tokenizer.batch_decode(outputs)[0])

<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
```<｜Assistant｜>```lean4
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  norm_num [abs_eq_self.mpr, abs_eq_neg_self.mpr, le_of_lt, sub_eq_add_neg, add_assoc]
  <;> norm_num
  <;> ring_nf
  <;> norm_num
  <;> apply abs_of_nonneg
  <;> norm_num
  <;> linarith
```<｜end▁of▁sentence｜>


In [18]:
prompt = """
Convert the following into lean statements:

if p, then if q, then r.

""".strip()

chat = [
  {"role": "user", "content": prompt},
]

inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

In [19]:
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


['<｜begin▁of▁sentence｜><｜User｜>Convert the following into lean statements:\n\nif p, then if q, then r.<｜Assistant｜>### Lean Statements\n\n1. **Statement**: If p, then if q, then r.\n   - **Lean Statement**: `p → (q → r)`\n\n2. **Statement**: If p and q, then r.\n   - **Lean Statement**: `p ∧ q → r`\n\n3. **Statement**: If p, then q, and if q, then r.\n   - **Lean Statement**: `p → q ∧ (q → r)`\n\n4. **Statement**: If p, then q, and if q, then r, and if r, then s.\n   - **Lean Statement**: `p → q ∧ (q → r ∧ (r → s))`\n\n5. **Statement**: If p, then q, and if q, then r, and if r, then s, and if s, then t.\n   - **Lean Statement**: `p → q ∧ (q → r ∧ (r → s ∧ (s → t)))`\n\n6. **Statement**: If p, then q, and if q, then r, and if r, then s, and if s, then t, and if t, then u.\n   - **Lean Statement**: `p → q ∧ (q → r ∧ (r → s ∧ (s → t ∧ (t → u))))`\n\n7. **Statement**: If p, then q, and if q, then r, and if r, then s, and if s, then t, and if t, then u, and if u, then v.\n   - **Lean Statem

In [20]:
print(tokenizer.batch_decode(outputs)[0])

<｜begin▁of▁sentence｜><｜User｜>Convert the following into lean statements:

if p, then if q, then r.<｜Assistant｜>### Lean Statements

1. **Statement**: If p, then if q, then r.
   - **Lean Statement**: `p → (q → r)`

2. **Statement**: If p and q, then r.
   - **Lean Statement**: `p ∧ q → r`

3. **Statement**: If p, then q, and if q, then r.
   - **Lean Statement**: `p → q ∧ (q → r)`

4. **Statement**: If p, then q, and if q, then r, and if r, then s.
   - **Lean Statement**: `p → q ∧ (q → r ∧ (r → s))`

5. **Statement**: If p, then q, and if q, then r, and if r, then s, and if s, then t.
   - **Lean Statement**: `p → q ∧ (q → r ∧ (r → s ∧ (s → t)))`

6. **Statement**: If p, then q, and if q, then r, and if r, then s, and if s, then t, and if t, then u.
   - **Lean Statement**: `p → q ∧ (q → r ∧ (r → s ∧ (s → t ∧ (t → u))))`

7. **Statement**: If p, then q, and if q, then r, and if r, then s, and if s, then t, and if t, then u, and if u, then v.
   - **Lean Statement**: `p → q ∧ (q → r ∧ 